<a href="https://colab.research.google.com/github/parthag1201/DocReversalEngine/blob/main/Workflow/WorkflowDiagramV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip install crewai_tools langchain_community langchain_google_genai fpdf2 markdown2

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install crewai agentic-ai

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Install the google-generativeai package
%pip install google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   --- ------------------------------------ 1.3/13.7 MB 7.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.6/13.7 MB 6.8 MB/s eta 0:00:02
   ----------- ---------------------------- 3.9/13.7 MB 6.8 MB/s eta 0:00:02
   ---------------- ----------------------- 5.5/13.7 MB 6.8 MB/s eta 0:00:02
   ------------------- -------------------- 6.8/13.7 MB 6.8 MB/s eta 0:00:02
   ------------------------ --------------- 8.4/13.7 MB 6.9 MB/s eta 0:00:01
   ------------------------------ --------- 10.5/13.7 MB 7.2 MB/s eta 0:00:01
   ----------------------------------- ---- 12.1/13.7 MB 7.3 MB/s eta 0:00:01
   --------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.8 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [14]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

c:\Users\akikumar\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# pdf read and write
from fpdf import FPDF
from markdown2 import Markdown

In [16]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Import crewAI library
from crewai import Agent, Task, Crew, Process
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List
from crewai.flow.flow import Flow, listen, start

In [18]:
import os

# %pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()

apiKey = os.getenv("GOOGLE_API_KEY")

# print(apiKey)

In [19]:
#setting up LLM

from crewai import LLM

llm = LLM(model="gemini/gemini-2.5-pro",
                             verbose=True,
                             temperature=0.5,
                             api_key = apiKey)

# print(apiKey)


In [21]:


# main.py
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
# from crewai_tools import ScrapeWebsiteTool
# --- Agent Definitions ---

# 1. ABAP Code Interpreter Agent
# This agent is the first point of contact with the raw ABAP code.
# Its primary goal is to understand the code's structure, identify key components
# like FORMs, FUNCTIONs, METHODs, and understand the high-level flow.
abap_interpreter = Agent(
    role='ABAP Code Interpreter',
    goal='Analyze the provided ABAP code to understand its structure, main components (like FORMs, METHODs, FUNCTIONs), and the overall program flow.',
    backstory=(
        "As a seasoned ABAP developer with decades of experience, you have an unparalleled ability to read and instantly "
        "comprehend even the most complex and archaic ABAP code. You can see beyond the syntax to the underlying business "
        "logic and program structure. Your task is to distill this complex code into a clear, structured summary that "
        "other agents can use."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 2. Logic Reviewer Agent
# This agent takes the structured summary from the interpreter and focuses on the
# business logic. It identifies conditional statements (IF/ELSE, CASE), loops (DO, WHILE),
# and subroutine calls to map out the decision points and processes.
logic_reviewer = Agent(
    role='Business Logic Analyst',
    goal='Review the structured summary of the ABAP code to identify and map out the core business logic, including conditional paths, loops, and subroutine calls.',
    backstory=(
        "You are a meticulous business analyst who specializes in reverse-engineering legacy systems. You have a keen eye for "
        "detail and can trace the flow of logic through complex code. Your strength is in identifying the decision points, "
        "data transformations, and processes that define how the program achieves its business objective. You translate "
        "technical flow into logical steps."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 3. Diagram Blocks Generator Agent
# This agent translates the logical steps identified by the reviewer into
# abstract diagram blocks. It decides what should be a process, a decision,
# an input/output, etc., without worrying about the final syntax.
diagram_blocks_generator = Agent(
    role='Workflow Diagram Architect',
    goal='Translate the identified business logic into a list of abstract diagram components (e.g., Start, End, Process, Decision, IO).',
    backstory=(
        "You are a systems architect who thinks visually. You can take a description of a process and immediately see it as a "
        "flowchart. You are an expert in UML and other diagramming methodologies, but your current focus is on defining the "
        "fundamental building blocks of the diagram—the nodes and their types—before they are rendered into a specific format."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 4. Diagram Generation Agent
# This agent takes the abstract blocks and generates the final diagram code
# in a specific format, in this case, MermaidJS.
# mermaid_docs_tool = ScrapeWebsiteTool(website_url='https://mermaid.js.org/intro/')

# Define the Mermaid Code Generator agent
diagram_generator = Agent(
    role='Mermaid Code Generator',
    goal='Generate syntactically correct Mermaid code for various diagrams from a natural language description.',
    backstory=(
        "You are an expert in creating diagrams-as-code using Mermaid. You have a deep understanding of Mermaid's syntax "
        "You know the latest syntax of MermaidJS and its common syntax errors and you have always have been known to give the perfect Mermaid code without any errors especially the common error of misuse of parantheses and brackets. "
        "for generating a wide variety of diagrams, including flowcharts, sequence diagrams, Gantt charts, and more. "
        "You can translate natural language descriptions or structured data into clean, readable, and accurate Mermaid code. "
        "You often consult the official Mermaid documentation to ensure you are using the latest and most effective features."
    ),
    # The tool that allows the agent to scrape/read websites
    # tools=[mermaid_docs_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm # Pass your configured LLM instance here
)


# 5. Manager Agent (Quality Control)
# This agent acts as the final quality gate. It reviews the generated diagram
# against the initial code to ensure it's a faithful representation—not too
# detailed (e.g., diagramming every single line) and not too high-level
# (e.g., missing critical logic branches).
manager = Agent(
    role='Quality Assurance Manager',
    goal='Review the final MermaidJS diagram to ensure it accurately represents the original ABAP code\'s logic without being overly complex or too simplistic. Provide the final, approved diagram code.',
    backstory=(
        "With a dual background in software development and project management, you are the ultimate quality gatekeeper. "
        "You ensure that the final product meets the initial requirements. Your job is to look at the generated diagram and the "
        "original code and ask: 'Does this diagram help a developer or analyst understand the code's workflow?' You have the final "
        "say on whether the diagram is approved."
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

## File Handling

In [10]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00


In [11]:
# 1. Read code from the 'code_files' folder
import PyPDF2 # Import PyPDF2 here
code_folder_path = 'code_files'
if not os.path.exists(code_folder_path):
    os.makedirs(code_folder_path)
    # Create a dummy file for demonstration if the folder is empty
    with open(os.path.join(code_folder_path, 'sample_code.txt'), 'w') as f:
        f.write("REPORT Z_SAMPLE_REPORT.") # Add some dummy code

abap_code_example = ""
for filename in os.listdir(code_folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(code_folder_path, filename), 'r') as f:
            abap_code_example += f.read() + "\n\n"

print(abap_code_example)

REPORT Z_SAMPLE_REPORT.




/usr/local/lib/python3.11/dist-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


In [ ]:
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import re

# Hide the root tkinter window
Tk().withdraw()

# Open file dialog
file_path = askopenfilename(
    title="Select ABAP Code File",
    filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")]
)

abap_code_example = ""

if file_path:
    with open(file_path, 'r') as f:
        abap_code_example = f.read()
    print("File selected:", file_path)
    print("Code Preview:\n", abap_code_example[:500])  # Show a preview

    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    sanitized_name = re.sub(r'[\\/*?:"<>|]', "_", base_filename)

else:
    print("No file selected.")

File selected: C:/Users/akikumar/Downloads/OneDrive_2025-07-28/FSTS Gemini/FS and TS/RTR/Code/1.ZGRTR_FI_HRC_CREDIT_UPD.txt
Code Preview:
 *&---------------------------------------------------------------------*
*& Report ZGRTR_FI_HRC_CREDIT_UPD
*&---------------------------------------------------------------------*
*&
*&---------------------------------------------------------------------*
REPORT zgrtr_fi_hrc_credit_upd.

*$-Top Include
INCLUDE zgrtr_fi_hrc_credit_top.

*$-Selection-screen
INCLUDE zgrtr_fi_hrc_credit_sel.

*$- Routines
INCLUDE zgrtr_fi_hrc_credit_f01.

* Begin of chg by Mohit on 26 Sep 2024
*$====================


In [23]:
# --- Task Definitions ---

# Example ABAP code to be processed.
# This code contains a simple report with a start-of-selection event,
# a loop, a conditional statement, and a form call.

# Task for the Interpreter
interpretation_task = Task(
    description=f"""
    Analyze the following ABAP code. Identify the main program events (like START-OF-SELECTION),
    loops (DO/WHILE), conditional logic (IF/ELSE), and subroutine calls (PERFORM).
    Create a clear, summarized list of these structural components and their sequence.

    ABAP Code:
    ---
    {abap_code_example}
    ---
    """,
    expected_output="A structured text summary outlining the main execution flow, loops, conditions, and subroutine calls in the ABAP code.",
    agent=abap_interpreter
)

# Task for the Logic Reviewer
logic_review_task = Task(
    description="""
    Based on the structured summary of the ABAP code, map out the business logic.
    Focus on the sequence of operations and the decisions being made.
    For example, describe the flow like: '1. Start. 2. Loop 5 times. 3. Inside loop, check if counter is even. 4. If even, do X. 5. If odd, do Y. 6. End loop. 7. End.'
    """,
    expected_output="A step-by-step description of the program's logic flow, highlighting decision points and repeated processes.",
    agent=logic_reviewer,
    context=[interpretation_task] # This task depends on the output of the interpretation_task
)

# Task for the Diagram Blocks Generator
block_generation_task = Task(
    description="""
    Convert the step-by-step logic flow into a list of abstract diagram blocks.
    Use standard flowchart notation:
    - 'Start' for the beginning
    - 'End' for the end
    - 'Process' for an action (e.g., 'Initialize variables', 'Write message')
    - 'Decision' for a condition (e.g., 'Is counter even?')
    - 'IO' for any input/output operations.
    - 'Loop' for start/end of loops.

    Define the connections between these blocks.
    """,
    expected_output="A list of objects or a structured text defining each diagram block (e.g., {id: 'A', type: 'Start', label: 'Start Program'}), and the connections between them (e.g., 'A --> B').",
    agent=diagram_blocks_generator,
    context=[logic_review_task]
)

# Task for the Diagram Generator
diagram_generation_task = Task(
    description="""
   Generate the Mermaid code for a flowchart based on a natural language description.
  The diagram should represent a simple user login process, with correct syntax for nodes and connections.
    The flowchart should start with a user entering their credentials.
    """,
    expected_output="A single code block containing the complete, syntactically correct  Mermaid code for the UML activity diagram.",
    agent=diagram_generator,
    context=[block_generation_task]
)

# Task for the Manager
manager_review_task = Task(
    description=f"""
    Review the generated MermaidJS code. Compare it against the original ABAP code's logic to ensure accuracy and appropriate detail.
    Do check the length and complexity of the diagram as it should neither be too small nor too large or complex.
    The diagram should clearly show the main loop, the conditional branch (even/odd check), and the subroutine call.
    It should not be cluttered with unnecessary details like variable declarations.
    If the diagram is accurate, provide the final MermaidJS code block as your final answer. If not, provide feedback (though for this run, assume it's correct).

    Original ABAP Code for reference:
    ---
    {abap_code_example}
    ---
    """,
    expected_output="The final, approved MermaidJS code block, ready for rendering.",
    agent=manager,
    context=[diagram_generation_task]
)

In [24]:
# --- Crew Definition ---

# Assemble the agents and tasks into a crew
abap_to_diagram_crew = Crew(
    agents=[abap_interpreter, logic_reviewer, diagram_blocks_generator, diagram_generator, manager],
    tasks=[interpretation_task, logic_review_task, block_generation_task, diagram_generation_task, manager_review_task],
    process=Process.sequential,  # The tasks will be executed one after another
    verbose= True  # Provides detailed logs of the process
)


# --- Execution ---

if __name__ == "__main__":
    print("🚀 Starting the ABAP to Diagram Crew with Gemini Pro...")
    print("======================================================")
    mermaid_code = abap_to_diagram_crew.kickoff(
    inputs={
        'abap_code': abap_code_example
    }
)

    print("\n========================================")
    print("✅ Crew execution finished!")
    print("\nFinal Result (MermaidJS Code):")
    print("----------------------------------------")
    # The final result is the output of the last task
    print(mermaid_code)
    print("----------------------------------------")

🚀 Starting the ABAP to Diagram Crew with Gemini Pro...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 36f454b9-0818-439f-ab57-852dce70f178                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: ABAP Code Interpreter                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the following ABAP code. Identify the main program events (like START-OF-SELECTION),               │
│      loops (DO/WHILE), conditional logic (IF/ELSE), and subroutine calls (PERFORM).                             │
│      Create a clear, summarized list of these structural components and their sequence.                         │
│                                                                                                                 │
│      ABAP Code:                                                                                                 │
│      ---                                                                                                        │
│      *&---------------------------------------------------------------------*                                   │
│  *& Report ZGRTR_FI_HRC_CREDIT_UPD                                                                              │
│  *&---------------------------------------------------------------------*                                       │
│  *&                                                                                                             │
│  *&---------------------------------------------------------------------*                                       │
│  REPORT zgrtr_fi_hrc_credit_upd.                                                                                │
│                                                                                                                 │
│  *$-Top Include                                                                                                 │
│  INCLUDE zgrtr_fi_hrc_credit_top.                                                                               │
│                                                                                                                 │
│  *$-Selection-screen                                                                                            │
│  INCLUDE zgrtr_fi_hrc_credit_sel.                                                                               │
│                                                                                                                 │
│  *$- Routines                                                                                                   │
│  INCLUDE zgrtr_fi_hrc_credit_f01.                                                                               │
│                                                                                                                 │
│  * Begin of chg by Mohit on 26 Sep 2024                                                                         │
│  *$=======================================================$*                                                    │
│  *$ At-Selection-Screen On Value Request                   *                                                    │
│  *$=======================================================$*                                                    │
│  *-F4 help for input file                                                                                       │
│  AT SELECTION-SCREEN ON VALUE-REQUEST FOR p_ppath.                                                              │
│    IF r_ps IS NOT INITIAL.                                                                                      │
│      PERFORM f4help_pres_server USING p_ppath.  "Presen

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: ABAP Code Interpreter                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### **Program Execution Analysis: ZGRTR_FI_HRC_CREDIT_UPD**                                                    │
│                                                                                                                 │
│  This document outlines the structure and execution flow of the ABAP report `ZGRTR_FI_HRC_CREDIT_UPD`. The      │
│  program is designed to read and process files to update credit data, with options for file locations on        │
│  either the presentation or application server.                                                                 │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **1. Program Structure & Components**                                                                      │
│                                                                                                                 │
│  The program is modularized using `INCLUDE` statements:                                                         │
│                                                                                                                 │
│  *   **`REPORT zgrtr_fi_hrc_credit_upd`**: The main program declaration.                                        │
│  *   **`INCLUDE zgrtr_fi_hrc_credit_top`**: Contains global data declarations (variables, structures, internal  │
│  tables).                                                                                                       │
│  *   **`INCLUDE zgrtr_fi_hrc_credit_sel`**: Defines the selection screen elements (`PARAMETERS`,                │
│  `SELECT-OPTIONS`, `BLOCKS`).                                                                                   │
│  *   **`INCLUDE zgrtr_fi_hrc_credit_f01`**: Contains the implementation of all `FORM` subroutines used in the   │
│  program.                                                                                                       │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **2. Program Execution Flow (Event Blocks)**                                                               │
│                                                                                                                 │
│  The program execution follows the standard ABAP report event sequence.                                         │
│                                                                                                                 │
│  **Event 1: `INITIALIZATION`**                                                                                  │
│  *   This block runs once when the program is loaded.                                                           │
│  *   **Action:** Sets the program title (`sy-title`) to 'Credit Data Update Engine'.                            │
│  *   **Action:** Assigns text literals to variables `s0

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 28cc149d-2086-4b02-b0a5-0f3b38a4f9c2                                                                     │
│  Agent: ABAP Code Interpreter                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by 
SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get 
local issuer certificate (_ssl.c:1028)')))

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {                         │
│    "error": {                                                                                                   │
│      "code": 503,                                                                                               │
│      "message": "The model is overloaded. Please try again later.",                                             │
│      "status": "UNAVAILABLE"                                                                                    │
│    }                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 3442594f-d9d0-45e7-a82c-7dbd96fdd88d                                                                     │
│  Agent: Workflow Diagram Architect                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 36f454b9-0818-439f-ab57-852dce70f178                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

InternalServerError: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}


In [ ]:
# Wrap the mermaid code in a minimal HTML with CDN
html_content = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Mermaid Diagram</title>
  <script type="module">
    import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
    mermaid.initialize({{ startOnLoad: true }});
  </script>
</head>
<body>
  <div class="mermaid">
    {mermaid_code}
  </div>
</body>
</html>
"""

# Save the HTML to a file in the current directory
html_file_path = os.path.join(os.path.dirname(file_path), sanitized_name + ".html")
with open(html_file_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"HTML file saved as: {html_file_path}")
